In [11]:
import numpy as np
import pandas as pd

# Context

Os dados utilizados neste trabalho são provenientes do KAGGLE: https://www.kaggle.com/c/novozymes-enzyme-stability-prediction
Cada amostra consiste numa sequência proteica, a sua termoestabilidade e o valor de pH ao qual a mesma foi medida. A previsão da termoestabilidade é fundamental no processo de engenharia de enzimas para um conjunto alargado de aplicações. A aplicação de metodologias de ML é uma excelente forma de atingir este objetivo, de forma a poupar tempo e dinheiro, relativamente a métodos laboratoriais.

# Preprocessing

In [2]:
train = pd.read_csv("Files/train.csv")
test = pd.read_csv("Files/test.csv")

updates = pd.read_csv("Files/train_updates_20220929.csv")

Definiram-se os dados de teste e treino e, ainda, se definiu dados de treino atualizados, de modo a corrigir-se alguns dos dados de treino.

In [3]:
train

,seq_id,protein_sequence,pH,data_source,tm
0,0,AAAAKAAALALLGEAPEVVDIWLPAGWRQPFRVFRLERKGDGVLVG...,7.0,doi.org/10.1038/s41592-020-0801-4,75.7
1,1,AAADGEPLHNEEERAGAGQVGRSLPQESEEQRTGSRPRRRRDLGSR...,7.0,doi.org/10.1038/s41592-020-0801-4,50.5
2,2,AAAFSTPRATSYRILSSAGSGSTRADAPQVRRLHTTRDLLAKDYYA...,7.0,doi.org/10.1038/s41592-020-0801-4,40.5
3,3,AAASGLRTAIPAQPLRHLLQPAPRPCLRPFGLLSVRAGSARRSGLL...,7.0,doi.org/10.1038/s41592-020-0801-4,47.2
4,4,AAATKSGPRRQSQGASVRTFTPFYFLVEPVDTLSVRGSSVILNCSA...,7.0,doi.org/10.1038/s41592-020-0801-4,49.5
...,...,...,...,...,...
31385,31385,YYMYSGGGSALAAGGGGAGRKGDWNDIDSIKKKDLHHSRGDEKAQG...,7.0,doi.org/10.1038/s41592-020-0801-4,51.8
31386,31386,YYNDQHRLSSYSVETAMFLSWERAIVKPGAMFKKAVIGFNCNVDLI...,7.0,doi.org/10.1038/s41592-020-0801-4,37.2
31387,31387,YYQRTLGAELLYKISFGEMPKSAQDSAENCPSGMQFPDTAIAHANV...,7.0,doi.org/10.1038/s41592-020-0801-4,64.6
31388,31388,YYSFSDNITTVFLSRQAIDDDHSLSLGTISDVVESENGVVAADDAR...,7.0,doi.org/10.1038/s41592-020-0801-4,50.7


In [4]:
updates

,seq_id,protein_sequence,pH,data_source,tm
0,69,NaN,NaN,NaN,NaN
1,70,NaN,NaN,NaN,NaN
2,71,NaN,NaN,NaN,NaN
3,72,NaN,NaN,NaN,NaN
4,73,NaN,NaN,NaN,NaN
...,...,...,...,...,...
2429,30738,NaN,NaN,NaN,NaN
2430,30739,NaN,NaN,NaN,NaN
2431,30740,NaN,NaN,NaN,NaN
2432,30741,NaN,NaN,NaN,NaN


In [5]:
print(f"Entries to update (num_rows): {updates['pH'].shape}")

mask = updates["pH"].isna()

to_delete = updates.loc[mask,:]
to_change = updates.loc[-mask,:]

print(f"Entries to delete (shape): {to_delete.shape}")
print(f"Entries to change (shape): {to_change.shape}")

Entries to update (num_rows): (2434,)
Entries to delete (shape): (2409, 5)
Entries to change (shape): (25, 5)


Verificou-se o número de dados para deletar e para modificar.

In [6]:
# First, change rows with data arrangement errors
train.loc[to_change.index, ["pH", "tm"]] = updates.loc[to_change.index, ["pH", "tm"]]train.loc[to_change.index, ["pH", "tm"]] = updates.loc[to_change.index, ["pH", "tm"]]

In [7]:
# Next, remove rows with data issues
print(f"Number of entries (original data): {train.shape[0]}")
train_cut = train.drop(to_delete.index)
print(f"Number of entries (after cut): {train_cut.shape[0]}")

Number of entries (original data): 31390
Number of entries (after cut): 28981


Após a realização das modificações foram eliminadas 2409 entradas.

In [8]:
train_cut

,seq_id,protein_sequence,pH,data_source,tm
25,25,AAPDEITTAWPVNVGPLNPHLYTPNQMFAQSMVYEPLVKYQADGSV...,7.0,doi.org/10.1038/s41592-020-0801-4,48.4
28,28,AARRFSGPRNQRQQGGGDPGLMHGKTVLITGANSGLGRATAAELLR...,7.0,doi.org/10.1038/s41592-020-0801-4,48.4
29,29,AASSPEADFVKKTISSHKIVIFSKSYCPYCKKAKSVFRELDQVPYV...,7.0,doi.org/10.1038/s41592-020-0801-4,49.0
30,30,AATFAYSQSQKRSSSSPGGGSNHGWNNWGKAAALASTTPLVHVASV...,5.5,doi.org/10.1038/s41592-020-0801-4,55.6
33,33,AAVLVTFIGGLYFITHHKKEESETLQSQKVTGNGLPPKPEERWRYI...,7.0,doi.org/10.1038/s41592-020-0801-4,48.4
...,...,...,...,...,...
31385,31385,YYMYSGGGSALAAGGGGAGRKGDWNDIDSIKKKDLHHSRGDEKAQG...,7.0,doi.org/10.1038/s41592-020-0801-4,51.8
31386,31386,YYNDQHRLSSYSVETAMFLSWERAIVKPGAMFKKAVIGFNCNVDLI...,7.0,doi.org/10.1038/s41592-020-0801-4,37.2
31387,31387,YYQRTLGAELLYKISFGEMPKSAQDSAENCPSGMQFPDTAIAHANV...,7.0,doi.org/10.1038/s41592-020-0801-4,64.6
31388,31388,YYSFSDNITTVFLSRQAIDDDHSLSLGTISDVVESENGVVAADDAR...,7.0,doi.org/10.1038/s41592-020-0801-4,50.7


Realizou-se a verificação das alterações realizadas aos dados de treino e a presença de NaNs -> verificou-se a existência de dados omissos (3494).

In [9]:
# check if there are still NaNs and remove them
idxs = train_cut[train_cut.isna().any(axis=1)].index
print(f"Number of rows still containing NaNs: {len(idxs)}")
train_cut.drop(index=idxs, inplace=True)
train_cut

Number of rows still containing NaNs: 3494


,seq_id,protein_sequence,pH,data_source,tm
25,25,AAPDEITTAWPVNVGPLNPHLYTPNQMFAQSMVYEPLVKYQADGSV...,7.0,doi.org/10.1038/s41592-020-0801-4,48.4
28,28,AARRFSGPRNQRQQGGGDPGLMHGKTVLITGANSGLGRATAAELLR...,7.0,doi.org/10.1038/s41592-020-0801-4,48.4
29,29,AASSPEADFVKKTISSHKIVIFSKSYCPYCKKAKSVFRELDQVPYV...,7.0,doi.org/10.1038/s41592-020-0801-4,49.0
30,30,AATFAYSQSQKRSSSSPGGGSNHGWNNWGKAAALASTTPLVHVASV...,5.5,doi.org/10.1038/s41592-020-0801-4,55.6
33,33,AAVLVTFIGGLYFITHHKKEESETLQSQKVTGNGLPPKPEERWRYI...,7.0,doi.org/10.1038/s41592-020-0801-4,48.4
...,...,...,...,...,...
31385,31385,YYMYSGGGSALAAGGGGAGRKGDWNDIDSIKKKDLHHSRGDEKAQG...,7.0,doi.org/10.1038/s41592-020-0801-4,51.8
31386,31386,YYNDQHRLSSYSVETAMFLSWERAIVKPGAMFKKAVIGFNCNVDLI...,7.0,doi.org/10.1038/s41592-020-0801-4,37.2
31387,31387,YYQRTLGAELLYKISFGEMPKSAQDSAENCPSGMQFPDTAIAHANV...,7.0,doi.org/10.1038/s41592-020-0801-4,64.6
31388,31388,YYSFSDNITTVFLSRQAIDDDHSLSLGTISDVVESENGVVAADDAR...,7.0,doi.org/10.1038/s41592-020-0801-4,50.7


Voltou-se a realizar a eliminação das linhas com NaNs.

In [10]:
# Reset indexes
train_cut.reset_index(drop=True, inplace=True)
train_cut

,seq_id,protein_sequence,pH,data_source,tm
0,25,AAPDEITTAWPVNVGPLNPHLYTPNQMFAQSMVYEPLVKYQADGSV...,7.0,doi.org/10.1038/s41592-020-0801-4,48.4
1,28,AARRFSGPRNQRQQGGGDPGLMHGKTVLITGANSGLGRATAAELLR...,7.0,doi.org/10.1038/s41592-020-0801-4,48.4
2,29,AASSPEADFVKKTISSHKIVIFSKSYCPYCKKAKSVFRELDQVPYV...,7.0,doi.org/10.1038/s41592-020-0801-4,49.0
3,30,AATFAYSQSQKRSSSSPGGGSNHGWNNWGKAAALASTTPLVHVASV...,5.5,doi.org/10.1038/s41592-020-0801-4,55.6
4,33,AAVLVTFIGGLYFITHHKKEESETLQSQKVTGNGLPPKPEERWRYI...,7.0,doi.org/10.1038/s41592-020-0801-4,48.4
...,...,...,...,...,...
25482,31385,YYMYSGGGSALAAGGGGAGRKGDWNDIDSIKKKDLHHSRGDEKAQG...,7.0,doi.org/10.1038/s41592-020-0801-4,51.8
25483,31386,YYNDQHRLSSYSVETAMFLSWERAIVKPGAMFKKAVIGFNCNVDLI...,7.0,doi.org/10.1038/s41592-020-0801-4,37.2
25484,31387,YYQRTLGAELLYKISFGEMPKSAQDSAENCPSGMQFPDTAIAHANV...,7.0,doi.org/10.1038/s41592-020-0801-4,64.6
25485,31388,YYSFSDNITTVFLSRQAIDDDHSLSLGTISDVVESENGVVAADDAR...,7.0,doi.org/10.1038/s41592-020-0801-4,50.7


Verificaram-se os parâmetros estatísticos para as colunas pH e tm. Analisando cada coluna observou-se que os valores médios e medianos são semelhates, que se elimanaram todos os valores omissos e que a distribuição nas duas colunas é relativamente uniforme (baixo skewness).

In [11]:
train_cut.describe().loc[:,["pH","tm"]]

,pH,tm
count,25487.000000,25487.000000
mean,6.903579,51.436933
std,0.752407,12.190382
min,1.990000,0.000000
25%,7.000000,43.650000
50%,7.000000,48.700000
75%,7.000000,54.500000
max,11.000000,130.000000


# Feature extraction

Seguimos com o passo da extração de features, com a esperança de obter um conjunto de descritores correlacionados com a variável dependente (termostabilidade), capazes de a prever com algoritmos adequados.

Para isto, utilizámos funções do BioPython e ProPy:

In [12]:
#!pip install propy3
#!conda install propy3

#### Descriptors

- sequence length (1 feature) -- <b>pypro3</b> (propy.PyPro)
- aminoacid composition (20 features) -- <b>pypro3</b> (propy.PyPro)
- dipeptide composition (400 features) -- <b>pypro3</b> (propy.PyPro)
- tripeptide composition (8000 features) -- <b>pypro3</b> (propy.PyPro)
- ctd descriptors -> composition, transition, distribution (147 features) -- <b>pypro3</b> (propy.PyPro)
- molecular weight (1 feature) -- <b>Biopython</b> (Bio.SeqUtils.ProtParam)
- aromaticity (1 feature) -- <b>Biopython</b> (Bio.SeqUtils.ProtParam)
- instability index (1 feature) -- <b>Biopython</b> (Bio.SeqUtils.ProtParam)
- isoelectric point (1 feature) -- <b>Biopython</b> (Bio.SeqUtils.ProtParam)
- secondary structure fraction -> helix, turn, sheet (3 features) -- <b>Biopython</b> (Bio.SeqUtils.ProtParam)
- molar extinction coefficient -> reduced, oxidized (2 features) -- <b>Biopython</b> (Bio.SeqUtils.ProtParam)
- geary autocorrelation descriptors (240 features) -- <b>pypro3</b> (propy.PyPro)
- moran autocorrelation descriptors (240 features) -- <b>pypro3</b> (propy.PyPro)
- normalized moreau-broto autocorrelation descriptors (240 features) -- <b>pypro3</b> (propy.PyPro)<br><br>

- Total number of features: <b>9297</b>


Extraímos inicialmente o tamanho de cada sequência (**'sequence length´**), seguido pela sua composição de aminoácidos individuais (**'aminoacid composition'**), e todos os dipéptidos ('dipeptide composition') e tripéptidos possíveis (**'tripeptide composition'**).

**'ctd descriptors'** são um conjunto de descritores relativamente às propriedades dos aminoácidos, como hidrofobicidade, polaridade, carga, etc. Para cada propriedade, os aminoácidos de cada proteína são agrupadas numa de 3 grupos, sendo calculadas a composição (percentagem de aminoácidos), transição (percentagem de transições de um grupo para outro), e distribuição (índices de cada categoria ao longo da sequência presentes em cada quartil são devolvidos e divididos pelo número total dessa categoria presente na sequência).

Seguem-se o peso molecular de cada proteína (**'molecular weight'**), frequência relativa de aminoácidos aromáticos (Phe, Trp e Tyr) (**'aromaticity'**), valor indicativo da instabilidade da proteína, onde valores altos significam proteínas mais instáveis (**'instability index'**), e o pH onde a carga global da proteína é neutra (**'isoelectric point'**).

**'secondary structure fraction'** refere-se à fração de aminoácidos na sequência que costumam estar presentes em hélices, 'turns' ou folhas. **'molar extinction coefficiente´** é uma medida do quão forte uma espécie química absorve luz numa dada onda de luz (considera cisteínas e pontes dissulfito).

Por fim, **'greary autocorrelation'**, **'moran autocorrelation'** e **'normalized moreau_broto autocorrelation'** são diferentes cálculos da distribuição espacial de um conjunto de descritores estruturais e fisioquímicos.

Para extrair estes descritores, foram criadas funções auxiliares para devolver os resultados em forma de dicionário. Desta forma é mais simples posteriormente juntar os resultados para criar um novo DataFrame do pandas.


#### Compute descriptors for all sequences

Criámos uma função que junta as funções anteriores todas, de forma a criar um novo dataset, juntando os descritores do dataset original (nomeadamente o 'pH' e 'tm') com os novos descritores inferidos a partir de cada sequência do dataset ('protein_sequence').

Uma vez que este processo demora bastante a correr, utilizámos a função 'tqdm' para visualizar o progresso da extração.

In [ ]:
from feature_extraction import get_dataset_with_features

train = get_dataset_with_features(train_cut)

Para facilitar o processo de recolha dos descritores sempre que se corre este notebook, guardou-se os resultados num ficheiro csv.

In [1]:
# export data_train to csv
train.to_csv("Files/data_train.csv")

In [7]:
train.head()

,seq_id,SeqLength,A,R,N,D,C,E,Q,G,...,MoreauBrotoAuto_Mutability23,MoreauBrotoAuto_Mutability24,MoreauBrotoAuto_Mutability25,MoreauBrotoAuto_Mutability26,MoreauBrotoAuto_Mutability27,MoreauBrotoAuto_Mutability28,MoreauBrotoAuto_Mutability29,MoreauBrotoAuto_Mutability30,pH,tm
0,25,501,10.379,4.192,4.591,5.589,0.000,6.387,5.389,4.990,...,-0.058,-0.057,-0.058,-0.056,-0.056,-0.056,-0.056,-0.057,7.0,48.4
1,28,313,7.987,7.668,4.473,3.834,2.236,7.029,3.834,9.904,...,0.019,0.017,0.014,0.017,0.014,0.012,0.013,0.010,7.0,48.4
2,29,109,7.339,3.670,1.835,6.422,1.835,9.174,2.752,8.257,...,0.087,0.079,0.068,0.051,0.065,0.059,0.050,0.048,7.0,49.0
3,30,329,7.295,3.343,6.991,7.599,0.304,6.079,3.343,8.815,...,0.074,0.077,0.074,0.073,0.074,0.078,0.081,0.082,5.5,55.6
4,33,278,10.432,7.554,2.878,2.158,0.719,8.273,15.108,5.396,...,0.057,0.051,0.053,0.052,0.050,0.050,0.051,0.049,7.0,48.4
